# Plot 2d histograms from quadrant analysis
## Plot multiple simulations at same height z/h

In [ ]:
%matplotlib inline
# import functions from python directory
import sys
sys.path.append("../python")
import os
import seaborn
import numpy as np
import xarray as xr
from matplotlib import pyplot as plt
from matplotlib import rc
from matplotlib.ticker import MultipleLocator
from LESnc import load_stats

In [ ]:
# plotting setup
rc('font',weight='normal',size=20,family='serif',serif='Times New Roman')
rc('text',usetex='True')
cmaps = [plt.get_cmap("Greys"), plt.get_cmap("Blues"), plt.get_cmap("Reds"), plt.get_cmap("Purples")]
cmap2 = seaborn.color_palette("cubehelix_r", as_cmap=True)

In [ ]:
# directories
# figure save
figdir = "/home/bgreene/SBL_LES/figures/quadrant/"
# list of simulations to loop over
sims = ["0.10", "0.25", "0.33", "0.50"]
# empty list to construct paths and store
dnc_all = []
for sim in sims:
    dnc = f"/home/bgreene/simulations/cr{sim}_u08_192/output/netcdf/"
    dnc_all.append(dnc)


In [ ]:
# loop over sims and load stats and quadrant data
sall = []
qall = []
quadall = []
for dnc in dnc_all:
    # load stats file
    s = load_stats(dnc+"average_statistics.nc")
    sall.append(s)
    # load data for 2d histogram
    q = xr.load_dataset(dnc+"u_w_theta_2d_quadrant.nc")
    qall.append(q)
    # load quadrant files
    quad = xr.load_dataset(dnc+"uw_tw_quadrant.nc")
    quadall.append(quad)

## Determine joint distributions
### u' - w'

In [ ]:
# initialize dictionary to store heights and simulations
H_uw_all = {}
# use same bins for each sim
uw_bins = (np.arange(-10., 10.1, 0.1), np.arange(-10., 10.1, 0.1))
# calculate bin area for normalizing later
uw_bin_area = np.diff(uw_bins[0])[0] * np.diff(uw_bins[1])[0]
# loop over heights and sims
for iz, sz in enumerate(np.around(qall[0].zh, 1)):
    sz = str(sz.values)
    H_uw_all[sz] = []
    for s, q in zip(sall, qall):
        # calculate bins and edges
        H_uw, x_uw, y_uw = np.histogram2d(q.u[iz].values/s.ustar0.values, 
                                          q.w[iz].values/s.ustar0.values, 
                                          bins=uw_bins, density=True)
        # normalize sum==1 by multiplying by bin area
        # also multiply by 100% for units
        H_uw *= uw_bin_area * 100.
        # append to big list
        H_uw_all[sz].append(H_uw)
# calculate bin centers
x1 = x_uw[:-1] + np.diff(x_uw)/2
y1 = y_uw[:-1] + np.diff(y_uw)/2

### Plot u' - w' joint dist
Subplots for each simulation and height

In [ ]:
fig1, ax1 = plt.subplots(nrows=len(H_uw_all), ncols=len(qall), 
                         sharex=True, sharey=True, figsize=(20, 14), 
                         constrained_layout=True)
# loop over H_uw_all and contour -- keys(height) and list elements(simulation)
# loop over heights (rows)
for iH, sH in enumerate(H_uw_all.keys()):
    # determine contour levels at each height
    # levels = get_contour_levels(H_uw_all[sH], 21)
    cax1 = []
    # loop over simulations (columns)
    for isim, sim in enumerate(H_uw_all[sH]):
        cax = ax1[iH,isim].contour(x1, y1, H_uw_all[sH][isim].T, cmap=cmap2,
                                   extend="both", levels=11)
        # ax1[iH,isim].clabel(cax, cax.levels, inline=True, fontsize=10)                           
        cax1.append(cax)
    # add colorbars at end of rows
    cb1 = fig1.colorbar(cax1[-1], ax=ax1[iH,:], location="right")
    cb1.ax.set_ylabel("Frequency [\\%]")

# add labels
[iax.set_xlabel("$u'/u_{*}$") for iax in ax1[-1,:]]
[iax.set_ylabel(f"$w'/u_{{*}}$, $z/h={{{sH}}}$") for sH, iax in zip(H_uw_all.keys(), ax1[:,0])]
ax1[0,0].set_xlim([-3, 3])
ax1[0,0].set_ylim([-2, 2])
[iax.axhline(0., c="k", alpha=0.5) for iax in ax1.flatten()]
[iax.axvline(0., c="k", alpha=0.5) for iax in ax1.flatten()]
[iax.set_title(f"$h/L = {{{(sall[i].he/sall[i].L).values:3.2f}}}$") for i, iax in enumerate(ax1[0, :])]

### $\theta'$ - $w'$

In [ ]:
# initialize dictionary to store heights and simulations
H_tw_all = {}
# use same bins for each sim
tw_bins = (np.arange(-15., 15.1, 0.1), np.arange(-10., 10.1, 0.1))
# calculate bin area for normalizing later
tw_bin_area = np.diff(tw_bins[0])[0] * np.diff(tw_bins[1])[0]
# loop over heights and sims
for iz, sz in enumerate(np.around(qall[0].zh, 1)):
    sz = str(sz.values)
    H_tw_all[sz] = []
    for s, q in zip(sall, qall):
        # calculate bins and edges
        H_tw, x_tw, y_tw = np.histogram2d(q.theta[iz].values/s.tstar0.values, 
                                          q.w[iz].values/s.ustar0.values, 
                                          bins=tw_bins, density=True)
        # normalize sum==1 by multiplying by bin area
        # also multiply by 100% for units
        H_tw *= tw_bin_area * 100.
        # append to big list
        H_tw_all[sz].append(H_tw)
# calculate bin centers
x2 = x_tw[:-1] + np.diff(x_tw)/2
y2 = y_tw[:-1] + np.diff(y_tw)/2

### Plot $\theta'$ - $w'$ joint

In [ ]:
fig2, ax2 = plt.subplots(nrows=len(H_tw_all), ncols=len(qall), 
                         sharex=True, sharey=True, figsize=(20, 14), 
                         constrained_layout=True)
# loop over H_uw_all and contour -- keys(height) and list elements(simulation)
# loop over heights (rows)
for iH, sH in enumerate(H_tw_all.keys()):
    cax2 = []
    # loop over simulations (columns)
    for isim, sim in enumerate(H_tw_all[sH]):
        cax = ax2[iH,isim].contour(x2, y2, H_tw_all[sH][isim].T, cmap=cmap2,
                                   extend="both", levels=11)
        # ax1[iH,isim].clabel(cax, cax.levels, inline=True, fontsize=10)                           
        cax2.append(cax)
    # add colorbars at end of rows
    cb2 = fig2.colorbar(cax2[-1], ax=ax2[iH,:], location="right")
    cb2.ax.set_ylabel("Frequency [\\%]")

# add labels
[iax.set_xlabel("$\\theta'/\\theta_{*}$") for iax in ax2[-1,:]]
[iax.set_ylabel(f"$w'/u_{{*}}$, $z/h={{{sH}}}$") for sH, iax in zip(H_tw_all.keys(), ax2[:,0])]
ax2[0,0].set_xlim([-5, 5])
ax2[0,0].set_ylim([-2, 2])
[iax.axhline(0., c="k", alpha=0.5) for iax in ax2.flatten()]
[iax.axvline(0., c="k", alpha=0.5) for iax in ax2.flatten()]
[iax.set_title(f"$h/L = {{{(sall[i].he/sall[i].L).values:3.2f}}}$") for i, iax in enumerate(ax2[0, :])]

### $\theta'$ - $u'$

In [ ]:
# initialize dictionary to store heights and simulations
H_tu_all = {}
# use same bins for each sim
tu_bins = (np.arange(-15., 15.1, 0.1), np.arange(-10., 10.1, 0.1))
# calculate bin area for normalizing later
tu_bin_area = np.diff(tu_bins[0])[0] * np.diff(tu_bins[1])[0]
# loop over heights and sims
for iz, sz in enumerate(np.around(qall[0].zh, 1)):
    sz = str(sz.values)
    H_tu_all[sz] = []
    for s, q in zip(sall, qall):
        # calculate bins and edges
        H_tu, x_tu, y_tu = np.histogram2d(q.theta[iz].values/s.tstar0.values, 
                                          q.u[iz].values/s.ustar0.values, 
                                          bins=tu_bins, density=True)
        # normalize sum==1 by multiplying by bin area
        # also multiply by 100% for units
        H_tu *= tu_bin_area * 100.
        # append to big list
        H_tu_all[sz].append(H_tu)
# calculate bin centers
x3 = x_tu[:-1] + np.diff(x_tu)/2
y3 = y_tu[:-1] + np.diff(y_tu)/2

In [ ]:
fig3, ax3 = plt.subplots(nrows=len(H_tu_all), ncols=len(qall), 
                         sharex=True, sharey=True, figsize=(20, 14), 
                         constrained_layout=True)
# loop over H_uw_all and contour -- keys(height) and list elements(simulation)
# loop over heights (rows)
for iH, sH in enumerate(H_tu_all.keys()):
    cax3 = []
    # loop over simulations (columns)
    for isim, sim in enumerate(H_tu_all[sH]):
        cax = ax3[iH,isim].contour(x3, y3, H_tu_all[sH][isim].T, cmap=cmap2,
                                   extend="both", levels=11)
        # ax1[iH,isim].clabel(cax, cax.levels, inline=True, fontsize=10)                           
        cax3.append(cax)
    # add colorbars at end of rows
    cb3 = fig2.colorbar(cax3[-1], ax=ax3[iH,:], location="right")
    cb3.ax.set_ylabel("Frequency [\\%]")

# add labels
[iax.set_xlabel("$\\theta'/\\theta_{*}$") for iax in ax3[-1,:]]
[iax.set_ylabel(f"$u'/u_{{*}}$, $z/h={{{sH}}}$") for sH, iax in zip(H_tu_all.keys(), ax3[:,0])]
ax3[0,0].set_xlim([-5, 5])
ax3[0,0].set_ylim([-3, 3])
[iax.axhline(0., c="k", alpha=0.5) for iax in ax3.flatten()]
[iax.axvline(0., c="k", alpha=0.5) for iax in ax3.flatten()]
[iax.set_title(f"$h/L = {{{(sall[i].he/sall[i].L).values:3.2f}}}$") for i, iax in enumerate(ax3[0, :])]

# Plot profiles of correlation coefficients $R_{uw}, R_{\theta w}, R_{\theta u}$

In [ ]:
# loop over simulations
for s, quad in zip(sall, quadall):
    # R_ab = <a'b'> / (sigma_a * sigma_b)
    s["Ruw"] = np.abs(s.uw_cov_tot / np.sqrt(s.u_var) / np.sqrt(s.w_var))
    s["Rtw"] = np.abs(s.tw_cov_tot / np.sqrt(s.theta_var) / np.sqrt(s.w_var))
    # eta_uw = <u'w'> / (u-w+ + u+w-)
    s["eta_uw"] = np.abs(s.uw_cov_tot) / np.abs((quad.uw_np + quad.uw_pn))
    # eta_tw = <theta'w'> / (t+w+ + t-w-)
    s["eta_tw"] = np.abs(s.tw_cov_tot) / np.abs((quad.tw_pp + quad.tw_nn))

In [ ]:
fig4, ax4 = plt.subplots(nrows=2, ncols=3, sharey=True, figsize=(14,10))
for s in sall:
    # Ruw
    ax4[0,0].plot(s.Ruw, s.z/s.he, label=f"$h/L={{{(s.he/s.L).values:3.2f}}}$")
    # Rtw
    ax4[0,1].plot(s.Rtw, s.z/s.he)
    # Ruw / Rtw
    ax4[0,2].plot(s.Ruw/s.Rtw, s.z/s.he)
    # eta_uw
    ax4[1,0].plot(s.eta_uw, s.z/s.he)
    # eta_tw
    ax4[1,1].plot(s.eta_tw, s.z/s.he)
    # eta_uw/eta_tw
    ax4[1,2].plot(s.eta_uw/s.eta_tw, s.z/s.he)

ax4[0,0].legend(fontsize=14)
ax4[0,0].set_xlim([0, 0.5])
ax4[1,0].set_xlim([0, 0.5])
ax4[0,1].set_xlim([0, 0.6])
ax4[1,1].set_xlim([0, 0.6])
ax4[0,2].set_xlim([0, 2])
ax4[1,2].set_xlim([0, 2])
ax4[1,0].set_ylim([0, 1])
ax4[0,0].set_xlabel("$R_{uw}$")
ax4[0,0].set_ylabel("$z/h$")
ax4[1,0].set_ylabel("$z/h$")
ax4[0,1].set_xlabel("$R_{\\theta w}$")
ax4[0,2].set_xlabel("$R_{uw}/R_{\\theta w}$")
ax4[1,0].set_xlabel("$\\eta_{uw} = \\overline{uw} / (\\overline{uw}^{II} + \\overline{uw}^{IV})$")
ax4[1,1].set_xlabel("$\\eta_{\\theta w} = \\overline{\\theta w} / (\\overline{\\theta w}^{I} + \\overline{\\theta w}^{III})$")
ax4[1,2].set_xlabel("$\\eta_{uw} / \\eta_{\\theta w}$")

In [ ]:
quad

# Profiles of quadrant fractions

In [ ]:
# u'w'
fig5, ax5 = plt.subplots(nrows=1, ncols=4, sharey=True, sharex=True, figsize=(14.8, 5))
# loop over simulations
for s, quad in zip(sall, quadall):
    # calc sum of quadrants -- NOTE: not weighted average!
    quad["uw_sum"] = np.abs(quad.uw_pp) + np.abs(quad.uw_pn) + np.abs(quad.uw_np) + np.abs(quad.uw_nn)
    # quadrant I: u+w+
    ax5[0].plot(np.abs(quad.uw_pp)/np.abs(quad.uw_sum), s.z/s.he, 
                label=f"$h/L={{{(s.he/s.L).values:3.2f}}}$")
    # quadrant II: u-w+
    ax5[1].plot(np.abs(quad.uw_np)/np.abs(quad.uw_sum), s.z/s.he)
    # quadrant III: u-w-
    ax5[2].plot(np.abs(quad.uw_nn)/np.abs(quad.uw_sum), s.z/s.he)
    # quadrant II: u+w-
    ax5[3].plot(np.abs(quad.uw_pn)/np.abs(quad.uw_sum), s.z/s.he)
# clean up
ax5[0].legend(fontsize=14)
ax5[0].set_ylim([0, 1])
ax5[0].set_ylabel("$z/h$")
ax5[0].set_xlim([0, 0.6])
[iax.set_xlabel(f"$|\\overline{{uw}}^{{{ii}}}| / \\Sigma | \\overline{{uw}}^k |$") for iax, ii in zip(ax5, ["I", "II", "III", "IV"])]
ax5[0].xaxis.set_major_locator(MultipleLocator(0.2))